In [42]:
# Practice problem from sync session 9
from sympy import *
from sympy.abc import *
from sympy.solvers import solve
import math

def full_simplify(expr):
    cur = expr
    while True:
        nxt = simplify(cur)
        if cur == nxt:
            return cur
        cur = nxt
        
def analyze_function(f, x, interval=[]):
    init_printing()
    d_f = diff(f, x, 1)
    critical_numbers = solve(d_f, x)
    print(f"------ Analysis for: {f} ------")
    print(f"limit as x approaches inf is: {limit(f, x, oo)}")
    print(f"limit as x approaches 0 is: {limit(f, x, 0)}")
    max_point = (None, -2**64)
    min_point = (None, 2**64)
    for cn in critical_numbers:
        if interval and cn > max(interval):
            continue
        if interval and cn < min(interval):
            continue
        point = (cn, f.subs(x, cn))
        max_point = max(max_point, point, key=lambda p: p[1])
        min_point = min(min_point, point, key=lambda p: p[1])
        print(f"Critical number {tuple(map(float, point))}")
   
    print("Interval ends are:")
    for n in interval:
        point = (n, (f.subs(x, n) if n is not oo else n))
        print(f"{point}")
        max_point = max(max_point, point, key=lambda p: p[1])
        min_point = min(min_point, point, key=lambda p: p[1])

    if max_point[0] != None or min_point[0] != None:
        print("Max/Min Points:")
        max_point[0] != None and print(f"Max point: {tuple(map(float, max_point))}")
        min_point[0] != None and print(f"Min point: {tuple(map(float, min_point))}")
    return full_simplify(f), full_simplify(d_f)

def second_derivative_test(f, x, y):
    f_x = f.diff(x)
    f_y = f.diff(y)
    f_xx = f.diff(x, 2)
    f_yy = f.diff(y, 2)
    f_xy = f.diff(x).diff(y)
    
    cps = solve([f_x, f_y])
    critical_points = cps if type(cps) == list else [cps]
    minimums = []
    maximums = []
    non_ms = []
    print("Critical numbers:")
    for cp in critical_points:
        print(cp)
        if not cp[x].is_real or not cp[y].is_real:
            continue
        D = f_xx.subs(x, cp[x]).subs(y, cp[y]) * \
            f_yy.subs(x, cp[x]).subs(y, cp[y]) - \
            pow(f_xy.subs(x, cp[x]).subs(y, cp[y]), 2)
        if D > 0 and f_xx.subs(x, cp[x]).subs(y, cp[y]) > 0:
            minimums.append((cp, f.subs(x, cp[x]).subs(y, cp[y]), D))
        elif D > 0 and f_xx.subs(x, cp[x]).subs(y, cp[y]) < 0:
            maximums.append((cp, f.subs(x, cp[x]).subs(y, cp[y]),D))
        elif D < 0:
            non_ms.append((cp, f.subs(x, cp[x]).subs(y, cp[y]),D))
    
    print(f"Minimums:")
    for m in sorted(minimums, key=lambda m: m[1]):
        print(f"f({m[0]}) = {m[1]}")   
    print(f"Maximums:")
    for m in reversed(sorted(maximums, key=lambda m: m[1])):
        print(f"f({m[0]}) = {m[1]}") 
        
    print(f"Saddle points:")
    for m in sorted(non_ms, key=lambda m: m[1]):
        print(f"f({m[0]}) = {m[1]}")          

def lagrange_optimize(f, g, vs):
    lam = Symbol('lam')
    eqs = [g]
    for v in vs:
        eqs.append(f.diff(v) - lam * g.diff(v))
    sls = solve(eqs)
    solutions = sls if type(sls) == list else [sls]
    print("Solutions:")
    for s in solutions:
        real_values = list(filter(lambda k: k.is_real, map(lambda u: s[u], vs)))
        if len(real_values) != len(vs):
            continue
        print("f(", end='')
        params = ''
        for v in vs:
            params += f"{s[v]},"
        print(params.strip(','), end='')
        print(") = ", end='')

        point = f
        for v in vs:
            point = point.subs(v, s[v])
        print(point)
        
# equation for a tagent plane
# z = f(a, b) + f_x(a, b)(x - a) + f_y(a, b)(y - b)
def tangent_plane(f, point, x, y):
    return f.subs(point) + \
            f.diff(x).subs(point) * (x - point[0][1]) + \
            f.diff(y).subs(point)* (y - point[1][1])

# total differntial dz = f_x(x, y) * dx + f_y(x, y) * dy
def approx(f, known_point, unknown_point):
    (x, x_val), (y, y_val) = known_point[0], known_point[1]
    dx = unknown_point[0][1] - x_val
    dy = unknown_point[1][1] - y_val
    dz = f.diff(x).subs(x, x_val).subs(y, y_val) * dx \
        + f.diff(y).subs(x, x_val).subs(y, y_val) * dy
    
    return f.subs(dict(known_point)) + dz, dz


In [46]:
def analyze_function(f, x, interval=[]):
    init_printing()
    d_f = diff(f, x, 1)
    critical_numbers = solve(d_f, x)
    print(f"------ Analysis for: {f} ------")
    max_point = (None, -2**64)
    min_point = (None, 2**64)
    print("Critical numbers:")
    for cn in critical_numbers:
        if interval and cn > max(interval):
            continue
        if interval and cn < min(interval):
            continue
        point = (cn, f.subs(x, cn))
        max_point = max(max_point, point, key=lambda p: p[1])
        min_point = min(min_point, point, key=lambda p: p[1])
        print(f"{tuple(map(float, point))}")
   
    print("Interval ends are:")
    for n in interval:
        point = (n, (f.subs(x, n) if n is not oo else n))
        print(f"{point}")
        max_point = max(max_point, point, key=lambda p: p[1])
        min_point = min(min_point, point, key=lambda p: p[1])

    if max_point[0] != None or min_point[0] != None:
        print("Max/Min Points:")
        max_point[0] != None and print(f"Max point: {tuple(map(float, max_point))}")
        min_point[0] != None and print(f"Min point: {tuple(map(float, min_point))}")
    return full_simplify(f), full_simplify(d_f)


S = -0.015*pow(x, 2) + 1.22*x - 7.5
analyze_function(S, x, [30, 60])
S

------ Analysis for: -0.015*x**2 + 1.22*x - 7.5 ------
Critical numbers:
(40.666666666666664, 17.306666666666665)
Interval ends are:
(30, 15.6000000000000)
(60, 11.7000000000000)
Max/Min Points:
Max point: (40.666666666666664, 17.306666666666665)
Min point: (60.0, 11.700000000000003)


In [10]:
expr = 0.1762*pow(x, 2) - 3.986*x + 22.68
a, b = 3, 9
expr, integrate(expr, (x, a, b))

In [11]:
from scipy.stats import norm
mu = 2.5
std = 0.2
zscore = lambda z: (z - mu)/std
1 - norm.cdf(zscore(2.7))

In [16]:
print(f"{mu - std*1.2} <= X <= {mu + std*1.2}")
norm.cdf(zscore(mu + std*1.2)) - norm.cdf(zscore(mu - std*1.2))

2.26 <= X <= 2.74


In [19]:
expon = lambda l, x: (1/l)*exp((1/-l)*x) 
integrate(expon(609.5, x), (x, 365, oo))

In [28]:
P = 1500 + 36*x - 1.5*pow(x, 2) + 120*y - 2*pow(y, 2)
second_derivative_test(P, x, y)

Critical numbers:
{x: 12.0000000000000, y: 30.0000000000000}
Minimums:
Maximums:
f({x: 12.0000000000000, y: 30.0000000000000}) = 3516.00000000000
Saddle points:


In [62]:
F = 3*pow(x, 1/3)*pow(y, 2/3)
G = 80*x + 150*y - 40000
lagrange_optimize(F, G, [x, y])
F, G

Solutions:
f(166.666666666667,177.777777777778) = 521.982352155848


In [34]:
G.subs(x, 166).subs(y, 178)

In [36]:
print(f"{F.subs(x, 166).subs(y, 178)}")
print(f"{F.subs(x, 167).subs(y, 177)}")

521.719759737351
520.805532815140


In [40]:
Z = pow(x, 0.65) * pow(y, 0.35)
approx(Z, [(x, 50), (y, 29)], [(x, 52), (y, 27)])

In [51]:
F = (11.5 - x) / 66
# Using sympy integration
print(f"P(X > 5) = {round(integrate(F, (x, 5, 11)), 4)}")
print(f"P(X < 4) = {round(integrate(F, (x, 0, 4)), 4)}")
print(f"P(7 <= X <= 8) = {round(integrate(F, (x, 7, 8)), 4)}")

P(X > 5) = 0.3182
P(X < 4) = 0.5758
P(7 <= X <= 8) = 0.0606


In [52]:
lam = 9
expon = lambda l, x: (1/l)*exp((1/-l)*x) 
# Using sympy integration
print(f"P(X >= 12) = {round(integrate(expon(lam, x), (x, 12, oo)), 4)}")
print(f"P(X < 2) = {round(integrate(expon(lam, x), (x, 0, 2)), 4)}")


P(X >= 12) = 0.2636
P(X < 2) = 0.1993


In [54]:
def second_derivative_test(f, x, y):
    f_x = f.diff(x)
    f_y = f.diff(y)
    f_xx = f.diff(x, 2)
    f_yy = f.diff(y, 2)
    f_xy = f.diff(x).diff(y)
    
    cps = solve([f_x, f_y])
    critical_points = cps if type(cps) == list else [cps]
    minimums = []
    maximums = []
    non_ms = []
    print("Critical numbers:")
    for cp in critical_points:
        print(cp)
        if not cp[x].is_real or not cp[y].is_real:
            continue
        D = f_xx.subs(x, cp[x]).subs(y, cp[y]) * \
            f_yy.subs(x, cp[x]).subs(y, cp[y]) - \
            pow(f_xy.subs(x, cp[x]).subs(y, cp[y]), 2)
        if D > 0 and f_xx.subs(x, cp[x]).subs(y, cp[y]) > 0:
            minimums.append((cp, f.subs(x, cp[x]).subs(y, cp[y]), D))
        elif D > 0 and f_xx.subs(x, cp[x]).subs(y, cp[y]) < 0:
            maximums.append((cp, f.subs(x, cp[x]).subs(y, cp[y]),D))
        elif D < 0:
            non_ms.append((cp, f.subs(x, cp[x]).subs(y, cp[y]),D))
    
    print(f"Minimums:")
    for m in sorted(minimums, key=lambda m: m[1]):
        print(f"f({m[0]}) = {m[1]}")   
    print(f"Maximums:")
    for m in reversed(sorted(maximums, key=lambda m: m[1])):
        print(f"f({m[0]}) = {m[1]}") 
        
    print(f"Saddle points:")
    for m in sorted(non_ms, key=lambda m: m[1]):
        print(f"f({m[0]}) = {m[1]}")          

L = pow(x, 2)*(3/2) + pow(y, 2) - 7*x - 6*y -2*x*y + 158
second_derivative_test(L, x, y)

Critical numbers:
{x: 13.0000000000000, y: 16.0000000000000}
Minimums:
f({x: 13.0000000000000, y: 16.0000000000000}) = 64.5000000000000
Maximums:
Saddle points:


In [75]:
def lagrange_optimize(f, g, vs):
    lam = Symbol('lam')
    eqs = [g]
    for v in vs:
        eqs.append(f.diff(v) - lam * g.diff(v))
    sls = solve(eqs)
    solutions = sls if type(sls) == list else [sls]
    print("Solutions:")
    for s in solutions:
        real_values = list(filter(lambda k: k.is_real, map(lambda u: s[u], vs)))
        if len(real_values) != len(vs):
            continue
        print("f(", end='')
        params = ''
        for v in vs:
            params += f"{v}: {s[v]},"
        print(params.strip(','), end='')
        print(") = ", end='')

        point = f
        for v in vs:
            point = point.subs(v, s[v])
        print(point)

h, r = symbols('h r') # height, radius
A = 2*math.pi*pow(r, 2) + h*2*math.pi*r # Area of cylindar = 2*{area of circle} + h*{circumference of circle}
V = math.pi*pow(r, 2)*h - 686*math.pi # Volume of cylindar
lagrange_optimize(A, V, [r, h])

Solutions:
f(r: 7.00000000000001,h: 14.0000000000000) = 923.628240155400


In [70]:
import math
math.pi